# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 8:04AM,247211,10,8565092,MedStone Pharma LLC,Released
1,Sep 26 2022 8:03AM,247210,10,8574237,"Snap Medical Industries, LLC",Released
2,Sep 26 2022 8:03AM,247210,10,8574238,"Snap Medical Industries, LLC",Released
3,Sep 26 2022 8:03AM,247210,10,8574239,"Snap Medical Industries, LLC",Released
4,Sep 26 2022 8:03AM,247210,10,8574240,"Snap Medical Industries, LLC",Released
5,Sep 26 2022 8:03AM,247210,10,8574241,"Snap Medical Industries, LLC",Released
6,Sep 26 2022 8:03AM,247210,10,8574242,"Snap Medical Industries, LLC",Released
7,Sep 26 2022 7:59AM,247209,10,Enova-10699,Emerginnova,Released
8,Sep 26 2022 7:59AM,247209,10,Eme-104470,Emerginnova,Released
9,Sep 26 2022 7:53AM,247208,22,628599,"NBTY Global, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
1,247207,Released,1
2,247208,Executing,1
3,247209,Released,2
4,247210,Released,6
5,247211,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247207,NaN,1.0
247208,1.0,NaN
247209,NaN,2.0
247210,NaN,6.0
247211,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
247206,0.0,12.0
247207,0.0,1.0
247208,1.0,0.0
247209,0.0,2.0
247210,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
247206,0,12
247207,0,1
247208,1,0
247209,0,2
247210,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,247206,0,12
1,247207,0,1
2,247208,1,0
3,247209,0,2
4,247210,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,247206,,12
1,247207,,1
2,247208,1,
3,247209,,2
4,247210,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC"
7,Sep 26 2022 7:59AM,247209,10,Emerginnova
9,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc."
10,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc."
11,Sep 26 2022 7:39AM,247206,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,,1
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC",,6
2,Sep 26 2022 7:59AM,247209,10,Emerginnova,,2
3,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc.",1,
4,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc.",,1
5,Sep 26 2022 7:39AM,247206,19,"Innogenix, LLC",,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,1,
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC",6,
2,Sep 26 2022 7:59AM,247209,10,Emerginnova,2,
3,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc.",,1
4,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc.",1,
5,Sep 26 2022 7:39AM,247206,19,"Innogenix, LLC",12,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,1,
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC",6,
2,Sep 26 2022 7:59AM,247209,10,Emerginnova,2,
3,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc.",,1
4,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,1.0,NaN
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC",6.0,NaN
2,Sep 26 2022 7:59AM,247209,10,Emerginnova,2.0,NaN
3,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc.",NaN,1.0
4,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 26 2022 8:04AM,247211,10,MedStone Pharma LLC,1.0,0.0
1,Sep 26 2022 8:03AM,247210,10,"Snap Medical Industries, LLC",6.0,0.0
2,Sep 26 2022 7:59AM,247209,10,Emerginnova,2.0,0.0
3,Sep 26 2022 7:53AM,247208,22,"NBTY Global, Inc.",0.0,1.0
4,Sep 26 2022 7:45AM,247207,19,"Methapharm, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,741630,9.0,0.0
19,494413,13.0,0.0
22,247208,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,741630,9.0,0.0
1,19,494413,13.0,0.0
2,22,247208,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,9.0,0.0
1,19,13.0,0.0
2,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,19,Released,13.0
2,22,Released,0.0
3,10,Executing,0.0
4,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,22
Status,,,
Executing,0.0,0.0,1.0
Released,9.0,13.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,22
0,Executing,0.0,0.0,1.0
1,Released,9.0,13.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,22
0,Executing,0.0,0.0,1.0
1,Released,9.0,13.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()